In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\kidney_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\kidney_disease_classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/proshanta000/kidney_disease_classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="proshanta000"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1d2afccec4c7566b7e8d9ed3f00a41e1e86ae8fd"

In [6]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any


@dataclass(frozen=True)
class EvaluationConfig:
    """
    Configuration for the model evaluation stage.
    """
    path_of_model: Path  # Path to the trained model to be evaluated
    validation_data: Path  # Path to the validation dataset
    mlflow_uri: str  # MLflow tracking server URI
    all_params: dict  # A dictionary containing all project parameters for logging
    params_image_size: list  # Image dimensions for evaluation
    params_batch_size: int  # Batch size for evaluation

In [7]:
import os
from pathlib import Path
from KidneyCNN.constants import *
from KidneyCNN.utils.common import (read_yaml, create_directories, save_json)


# This class manages the configuration settings for the entire project pipeline.
# It reads settings from 'config.yaml' and 'params.yaml' and provides them
# to each pipeline stage in a structured format.
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initializes the ConfigurationManager by loading configuration and
        parameters from YAML files.
        """
        # Load the main configuration file
        self.config = read_yaml(config_filepath)
        # Load the model hyperparameters
        self.params = read_yaml(params_filepath)

        # Create the root artifacts directory if it doesn't exist
        create_directories([self.config.artifacts_root])

    
    

    def get_evalution_config(self) -> EvaluationConfig:
        """
        Provides the configuration for the model evaluation stage.
        """
        # Define the path to the validation data directory
        validation_data = os.path.join(self.config.data_transformation.data_split_dir,  "val")
        
        # Create an EvaluationConfig object with the relevant settings
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5", # Path to the trained model
            validation_data=Path(validation_data), # Path to the validation data
            mlflow_uri="https://dagshub.com/proshanta000/kidney_disease_classification.mlflow", # MLflow tracking URI
            all_params= self.params, # All model parameters for logging to MLflow
            params_image_size= self.params.IMAGE_SIZE, # Image size for evaluation
            params_batch_size= self.params.BATCH_SIZE # Batch size for evaluation
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import json

In [9]:
class Evalution:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        # We assume the data is already pre-split into train/valid directories.
        # Therefore, we do not need the validation_split parameter.
        datagenerator_kwargs = dict(
            rescale=1. / 255
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        score = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=score)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Log all parameters, including nested ones
            mlflow.log_params(self.config.all_params)

            # Log each metric individually
            mlflow.log_metric(
                key="loss",
                value=self.score[0]
            )
            mlflow.log_metric(
                key="accuracy",
                value=self.score[1]
            )

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model",
                                       registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evalution_config()
    evaluation= Evalution(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-10-05 01:36:52,977: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-05 01:36:52,984: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-05 01:36:52,989: INFO: common: created directory at: artifacts]
Found 900 images belonging to 4 classes.
57/57 [==============================] - 199s 3s/step - loss: 12.8280 - accuracy: 0.6144
[2025-10-05 01:40:13,069: INFO: common: json file saved at: scores.json]


2025/10/05 01:40:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-10-05 01:40:17,466: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2025-10-05 01:40:18,259: INFO: builder_impl: Assets written to: C:\Users\prosh\AppData\Local\Temp\tmp2wh3vr28\model\data\model\assets]


d:\Data_science\Projects\kidney_disease_classification\venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/10/05 01:42:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
